In [4]:
import os

import numpy as np
import pandas as pd
from PIL import Image
from tqdm import tqdm
from vllm import LLM, SamplingParams

# Custom libraries
from vqa_dataset import PromptDataset,prompt_collate,create_template

In [5]:
## User Input ##
model       = "Qwen/Qwen2-VL-2B-Instruct"
task        = "classifcation"
save_every  = 50
options     = True
out_dit     = "out"
model_dir   = "/pasteur/u/rdcunha/models"

## Envs:
os.environ["HF_HOME"] = model_dir
os.environ["TRANSFORMERS_CACHE"] = model_dir
os.environ["HUGGINGFACE_HUB_CACHE"] = model_dir
os.environ["VLLM_CACHE_ROOT"]  = model_dir
os.environ["TOKENIZERS_PARALLELISM"] = "false"
os.environ["CUDA_VISIBLE_DEVICES"] = "1"

Image.MAX_IMAGE_PIXELS = None

In [6]:
llm = LLM(
    hf_overrides= {"cache_dir": model_dir},
    download_dir=model_dir,
    model=model,
    trust_remote_code=True,       # required for LLaVA
    tensor_parallel_size=1,       # >1 if you have multiple GPUs
    gpu_memory_utilization=0.95,  # pack VRAM aggressively
    max_model_len=4096,           # tune to your context needs
)

INFO 11-10 18:31:23 [utils.py:233] non-default args: {'trust_remote_code': True, 'download_dir': '/pasteur/u/rdcunha/models', 'max_model_len': 4096, 'gpu_memory_utilization': 0.95, 'disable_log_stats': True, 'hf_overrides': {'cache_dir': '/pasteur/u/rdcunha/models'}, 'model': 'Qwen/Qwen2-VL-2B-Instruct'}


The argument `trust_remote_code` is to be used with Auto classes. It has no effect here and is ignored.


INFO 11-10 18:31:23 [model.py:547] Resolved architecture: Qwen2VLForConditionalGeneration
INFO 11-10 18:31:23 [model.py:1510] Using max model len 4096
INFO 11-10 18:31:23 [scheduler.py:205] Chunked prefill is enabled with max_num_batched_tokens=8192.
(EngineCore_DP0 pid=108096) INFO 11-10 18:31:24 [core.py:644] Waiting for init message from front-end.
(EngineCore_DP0 pid=108096) INFO 11-10 18:31:24 [core.py:77] Initializing a V1 LLM engine (v0.11.0) with config: model='Qwen/Qwen2-VL-2B-Instruct', speculative_config=None, tokenizer='Qwen/Qwen2-VL-2B-Instruct', skip_tokenizer_init=False, tokenizer_mode=auto, revision=None, tokenizer_revision=None, trust_remote_code=True, dtype=torch.bfloat16, max_seq_len=4096, download_dir='/pasteur/u/rdcunha/models', load_format=auto, tensor_parallel_size=1, pipeline_parallel_size=1, data_parallel_size=1, disable_custom_all_reduce=False, quantization=None, enforce_eager=False, kv_cache_dtype=auto, device_config=cuda, structured_outputs_config=Structured

chat_template.json: 0.00B [00:00, ?B/s]

(EngineCore_DP0 pid=108096) INFO 11-10 18:31:29 [gpu_model_runner.py:2602] Starting to load model Qwen/Qwen2-VL-2B-Instruct...
(EngineCore_DP0 pid=108096) INFO 11-10 18:31:29 [gpu_model_runner.py:2634] Loading model from scratch...
(EngineCore_DP0 pid=108096) INFO 11-10 18:31:29 [cuda.py:366] Using Flash Attention backend on V1 engine.
(EngineCore_DP0 pid=108096) INFO 11-10 18:31:30 [weight_utils.py:392] Using model weights format ['*.safetensors']


...

model.safetensors.index.json: 0.00B [00:00, ?B/s]

Loading safetensors checkpoint shards:   0% Completed | 0/2 [00:00<?, ?it/s]


(EngineCore_DP0 pid=108096) INFO 11-10 18:33:11 [default_loader.py:267] Loading weights took 38.24 seconds
(EngineCore_DP0 pid=108096) INFO 11-10 18:33:12 [gpu_model_runner.py:2653] Model loading took 4.1513 GiB and 102.282610 seconds
(EngineCore_DP0 pid=108096) INFO 11-10 18:33:13 [gpu_model_runner.py:3344] Encoder cache will be initialized with a budget of 16384 tokens, and profiled with 1 image items of the maximum feature size.
(EngineCore_DP0 pid=108096) INFO 11-10 18:33:30 [backends.py:548] Using cache directory: /pasteur/u/rdcunha/models/torch_compile_cache/bbeb05e7d5/rank_0_0/backbone for vLLM's torch.compile
(EngineCore_DP0 pid=108096) INFO 11-10 18:33:30 [backends.py:559] Dynamo bytecode transform time: 7.41 s
(EngineCore_DP0 pid=108096) INFO 11-10 18:33:33 [backends.py:197] Cache the graph for dynamic shape for later use
(EngineCore_DP0 pid=108096) INFO 11-10 18:33:48 [backends.py:218] Compiling a graph for dynamic shape takes 17.02 s
(EngineCore_DP0 pid=108096) INFO 11-10 1

Capturing CUDA graphs (mixed prefill-decode, PIECEWISE): 10
Capturing CUDA graphs (decode, FULL): 100%|█| 35/35 [00:01<


(EngineCore_DP0 pid=108096) INFO 11-10 18:33:58 [gpu_model_runner.py:3480] Graph capturing finished in 5 secs, took 2.76 GiB
(EngineCore_DP0 pid=108096) INFO 11-10 18:33:58 [core.py:210] init engine (profile, create kv cache, warmup model) took 46.07 seconds
INFO 11-10 18:33:59 [llm.py:306] Supported_tasks: ['generate']


In [7]:
message = {
    "role": "user",
    "content": [
        {"type": "text", "text": "Describe this set of frames. Consider the frames to be a part of the same video."},
    ],
}

outputs = llm.chat([message])

for o in outputs:
    generated_text = o.outputs[0].text
    print(generated_text)

The image processor of type `Qwen2VLImageProcessor` is now loaded as a fast processor by default, even if the model checkpoint was saved with a slow processor. This is a breaking change and may produce slightly different outputs. To continue using the slow processor, instantiate this class with `use_fast=False`. Note that this behavior will be extended to all models in a future release.


INFO 11-10 18:46:09 [chat_utils.py:560] Detected the chat template content format to be 'openai'. You can set `--chat-template-content-format` to override this.
WARNING 11-10 18:46:09 [model.py:1389] Default sampling parameters have been overridden by the model's Hugging Face generation config recommended from the model creator. If this is not intended, please relaunch vLLM instance with `--generation-config vllm`.


Adding requests:   0%|          | 0/1 [00:00<?, ?it/s]

Processed prompts:   0%| | 0/1 [00:00<?, ?it/s, est. speed 

I'm sorry, but you haven't provided any frames for me to describe.


In [25]:
task     = "all_cls"
mbu_root = f"/pasteur/u/rdcunha/data_cache/mmbu/final_data/VLMEvalData_v2/LMUData/{task}"

data_root= os.path.join(mbu_root, 'all_cls_closed_subsampled.tsv')

data = pd.read_csv(data_root,sep='\t')
filtered_ = data[data["question_type"] == "expert"]
df_ = filtered_[~filtered_["dataset"].isin([ "isic2018",'herlev',"breakhis_400x","breakhis_200x"])]

In [26]:
# quality tests
#print(f"Datasets:{filtered_["dataset"].unique()}")
no_opitons_d = 0
for options in filtered_["options"].unique():
    #print(options)
    if len(options.split(","))  < 2: no_opitons_d +=1
    else:pass

print(no_opitons_d)

0


In [27]:
from torch.utils.data import Dataset, DataLoader
ds = PromptDataset(df=df_)
questions_data_loaders = DataLoader(ds, 
                                    batch_size=10, 
                                    shuffle=False,
                                    collate_fn=prompt_collate, 
                                    num_workers=10,
                                    persistent_workers=True, 
                                    pin_memory=True, 
                                    prefetch_factor=4)


# Test

In [28]:
sampling_params = SamplingParams(temperature=0, max_tokens=512)
for items in tqdm(questions_data_loaders, desc="Processing batches"):

    # Generate only for new items
    conversations = [create_template(it) for it in items]
    outputs = llm.generate(conversations, sampling_params=sampling_params)
    break

Processing batches:   0%|          | 0/735 [00:00<?, ?it/s]

Adding requests:   0%|          | 0/10 [00:00<?, ?it/s]

Processed prompts:   0%| | 0/10 [00:00<?, ?it/s, est. speed

Processing batches:   0%|          | 0/735 [00:12<?, ?it/s]


# Run

In [34]:
import os
import json
from tqdm import tqdm
model_name = model
try:
    model_name = model_name.split('/')[1]
except:
    model_name = model_name
    
save_path = f"{model_name}_{task}_expert_closed.jsonl"
save_file = os.path.join(out_dit, save_path)

# --- Step 1: Collect already processed IDs ---
existing_ids = set()
if os.path.exists(save_file):
    with open(save_file, "r") as f:
        for line in f:
            try:
                data = json.loads(line)
                existing_ids.add(data["index"])
            except json.JSONDecodeError:
                continue  # skip corrupted lines

print(f"Found {len(existing_ids)} already processed items. Skipping them...")

# --- Step 2: Run inference only for new IDs ---
saved_items = []
counter = 0
sampling_params = SamplingParams(temperature=0, max_tokens=512)

with open(save_file, "a") as f:
    for items in tqdm(questions_data_loaders, desc="Processing batches"):
        # Filter out items whose IDs already exist
        new_items = [it for it in items if it["index"] not in existing_ids]
        if not new_items:
            continue  # nothing new in this batch

        ### THIS USUALLY NEEDS TO BE EDITED ###
        messages = [create_template(item) for item in items]
        inputs = processor.apply_chat_template( messages, add_generation_prompt=True, padding=True, tokenize=True, return_dict=True, return_tensors="pt").to(model.device, dtype=torch.bfloat16)
        input_len = inputs["input_ids"].shape[-1]
        try:
            with torch.inference_mode():
                generation = model.generate(**inputs, max_new_tokens=100, do_sample=False)
                outputs = []
                for g in generation:
                    g = g[input_len:]
                    decoded = processor.decode(g, skip_special_tokens=True)
                    outputs.append(decoded)
        except: 
            print(f"could not generate for {items}")
            continue
        
            

            #####
       
        #except:
        #    print(f"could not generate for {items}")
        #    continue
       ### THIS USUALLY NEEDS TO BE EDITED ###

        for it, output in zip(new_items, outputs):
           
    
            for it, output in zip(new_items, outputs):
                answer = output.outputs[0].text
                saved_items.append({
                    "index": it["index"],
                    "question": it["question"],
                    "options": it["options"],
                    "image_path": it["image_path"],
                    "image_scale": it["image_scale"],
                    "scaled_width": it["scaled_width"],
                    "scaled_height": it["scaled_height"],
                    "dataset": it["dataset"],
                    "answer": answer
                })
            existing_ids.add(it["index"])  # add to skip list in case of crash recovery
            counter += 1

            # Save every N examples
            if counter % save_every == 0:
                print(f"Saving at {save_file}")
                for s in saved_items:
                    f.write(json.dumps(s) + "\n")
                f.flush()
                saved_items = []

        #print("Could not run batch:",items)
    # Save remaining items
    if saved_items:
        for s in saved_items:
            f.write(json.dumps(s) + "\n")


Found 0 already processed items. Skipping them...


Processing batches:   0%|          | 0/735 [00:00<?, ?it/s]


NameError: name 'processor' is not defined

The history saving thread hit an unexpected error (OperationalError('database is locked')).History will not be written to the database.


In [37]:
import os
import json
from tqdm import tqdm

model_name = model
try:
    model_name = model_name.split('/')[1]
except:
    model_name = model_name
    
save_path = f"{model_name}_{task}_expert_closed.jsonl"
save_file = os.path.join(out_dit, save_path)

# --- Step 1: Collect already processed IDs ---
existing_ids = set()
if os.path.exists(save_file):
    with open(save_file, "r") as f:
        for line in f:
            try:
                data = json.loads(line)
                existing_ids.add(data["index"])
            except json.JSONDecodeError:
                continue  # skip corrupted lines

print(f"Found {len(existing_ids)} already processed items. Skipping them...")

# --- Step 2: Run inference only for new IDs ---
saved_items = []
counter = 0
sampling_params = SamplingParams(temperature=0, max_tokens=512)

with open(save_file, "a") as f:
    for items in tqdm(questions_data_loaders, desc="Processing batches"):
        # Filter out items whose IDs already exist
        new_items = [it for it in items if it["index"] not in existing_ids]
        if not new_items:
            continue  # nothing new in this batch

        ### THIS USUALLY NEEDS TO BE EDITED ###
     
        try:
            conversations = [create_template(it) for it in new_items]
            outputs = llm.generate(conversations, sampling_params=sampling_params)
           
        except: 
            print(f"could not generate for {items}")
            continue
        
            

            #####
       
        #except:
        #    print(f"could not generate for {items}")
        #    continue
       ### THIS USUALLY NEEDS TO BE EDITED ###

        for it, output in zip(new_items, outputs):
            answer = output.outputs[0].text
            saved_items.append({
                "index": it["index"],
                "question": it["question"],
                "options": it["options"],
                "image_path": it["image_path"],
                "image_scale": it["image_scale"],
                "scaled_width": it["scaled_width"],
                "scaled_height": it["scaled_height"],
                "dataset": it["dataset"],
                "class_label":it["class_label"],
                
                "answer": answer
            })
            existing_ids.add(it["index"])  # add to skip list in case of crash recovery
            counter += 1

            # Save every N examples
            if counter % save_every == 0:
                print(f"Saving at {save_file}")
                for s in saved_items:
                    f.write(json.dumps(s) + "\n")
                f.flush()
                saved_items = []

        #print("Could not run batch:",items)
    # Save remaining items
    if saved_items:
        for s in saved_items:
            f.write(json.dumps(s) + "\n")


Found 0 already processed items. Skipping them...


Processing batches:   0%|          | 0/735 [00:00<?, ?it/s]

Adding requests:   0%|          | 0/10 [00:00<?, ?it/s]

Processed prompts:   0%| | 0/10 [00:00<?, ?it/s, est. speed

Processing batches:   0%|  | 1/735 [00:00<02:16,  5.38it/s]

...

Processed prompts:   0%| | 0/3 [00:00<?, ?it/s, est. speed 

Processing batches:  99%|▉| 731/735 [12:16<00:05,  1.40s/it

Adding requests:   0%|          | 0/1 [00:00<?, ?it/s]

Processed prompts:   0%| | 0/1 [00:00<?, ?it/s, est. speed 

Processing batches: 100%|█| 735/735 [12:16<00:00,  1.00s/it
